# Process Framework: Malware Analysis Pipeline with HITL

This notebook demonstrates how to build a production-grade **Malware Analysis Pipeline** using the **Semantic Kernel Process Framework**.

We will build a multi-step workflow that combines deterministic logic (static analysis) with AI capabilities (classification and remediation), and finally add a **Human-in-the-Loop (HITL)** approval gate for safety.

## Scenario Overview

We are building a system to analyze potentially suspicious files:

1.  **Step 1: Static Analysis (No AI)**
    *   Inspects file metadata (extension, size) and checks for suspicious strings.
    *   Output: `StaticAnalysisResult`

2.  **Step 2: Decision Point (No AI)**
    *   Routes the process based on static analysis results.
    *   **If Suspicious** → Proceed to AI Classification.
    *   **If Not Suspicious** → Skip AI, go directly to Remediation (Low Risk).

3.  **Step 3: AI Malware Classification**
    *   Uses an LLM to analyze the static summary and determine a verdict (Malicious/Benign/Unknown).
    *   Output: `MalwareClassification`

4.  **Step 4: AI Remediation Guidance**
    *   Generates operational steps for the security team.
    *   Can be triggered by either the Static Analysis (low risk) or the AI Classification (high risk).

### High-Level Architecture

```mermaid
graph TD
    Start(AnalyzeFile) --> Static[Step 1: Static Analysis]
    Static --> Decision{Step 2: Decision}
    Decision -- Suspicious --> AIClass[Step 3: AI Classification]
    Decision -- Not Suspicious --> Remediation[Step 4: Remediation]
    AIClass --> Remediation
```

## Setup and Configuration

In [34]:
#r "nuget: Microsoft.SemanticKernel, 1.32.0"

#!import config/Settings.cs

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId, embeddingEndpoint, embeddingApiKey) = Settings.LoadFromFile();

Console.WriteLine($"Configuration loaded:");
Console.WriteLine($"- Model: {model}");
Console.WriteLine($"- Endpoint: {azureEndpoint}");

Installed Packages Microsoft.SemanticKernel, 1.32.0

Configuration loaded:
- Model: gpt-4o
- Endpoint: https://eastus.api.cognitive.microsoft.com/


In [35]:
// Install Semantic Kernel packages
#r "nuget: Microsoft.SemanticKernel, 1.32.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.32.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Process.Abstractions, 1.32.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Process.Core, 1.32.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Process.LocalRuntime, 1.32.0-alpha"

using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel; 
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Process;
using System.ComponentModel;
using System.Text.Json;

// Suppress experimental API warnings
#pragma warning disable SKEXP0080
#pragma warning disable SKEXP0001

Console.WriteLine("Packages loaded successfully.");

Installed Packages Microsoft.SemanticKernel, 1.32.0 Microsoft.SemanticKernel.Agents.Core, 1.32.0-alpha Microsoft.SemanticKernel.Process.Abstractions, 1.32.0-alpha Microsoft.SemanticKernel.Process.Core, 1.32.0-alpha Microsoft.SemanticKernel.Process.LocalRuntime, 1.32.0-alpha

Packages loaded successfully.


In [36]:
// Initialize the Kernel
var builder = Kernel.CreateBuilder();
if (useAzureOpenAI)
{
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
}
else
{
    builder.AddOpenAIChatCompletion(model, apiKey);
}
Kernel kernel = builder.Build();

Console.WriteLine("Kernel initialized.");

Kernel initialized.


## Part 1: Defining the Data Models and Events

First, we define the data structures that will be passed between our process steps.

In [37]:
public sealed class StaticAnalysisResult
{
    public string FilePath { get; set; }
    public long FileSizeBytes { get; set; }
    public bool IsExecutable { get; set; }
    public bool ContainsSuspiciousStrings { get; set; }
    public string Summary { get; set; }
    
    // Deterministic logic for suspicion
    public bool IsSuspicious => IsExecutable || ContainsSuspiciousStrings || FileSizeBytes > 50_000_000;
}

public sealed class MalwareClassification
{
    public string FilePath { get; set; }
    public string Verdict { get; set; }       // Malicious / Benign / Unknown
    public string Confidence { get; set; }    // Low / Medium / High
    public string Reasoning { get; set; }
}

public static class ProcessEvents
{
    public const string AnalyzeFile = nameof(AnalyzeFile);
    public const string StaticAnalysisCompleted = nameof(StaticAnalysisCompleted);
    public const string NeedsAIAnalysis = nameof(NeedsAIAnalysis);
    public const string SkipAIAnalysis = nameof(SkipAIAnalysis);
    public const string AIClassified = nameof(AIClassified);
    public const string RemediationGenerated = nameof(RemediationGenerated);
    
    // HITL Events
    public const string HumanApprovalRequired = nameof(HumanApprovalRequired);
    public const string HumanVerified = nameof(HumanVerified);
    public const string HumanRejected = nameof(HumanRejected);
}

## Part 2: Implementing the Process Steps

We will now implement the four core steps of our pipeline.

In [38]:
#pragma warning disable SKEXP0080

// Step 1: Static Analysis (No AI)
public class StaticAnalysisStep : KernelProcessStep
{
    public static class Functions { public const string RunStaticAnalysis = nameof(RunStaticAnalysisAsync); }

    [KernelFunction(Functions.RunStaticAnalysis)]
    public async Task RunStaticAnalysisAsync(KernelProcessStepContext context, string filePath)
    {
        Console.WriteLine($"\n[StaticAnalysis] Analyzing file: {filePath}");

        // Simulate file analysis logic
        var result = new StaticAnalysisResult
        {
            FilePath = filePath,
            FileSizeBytes = 1024 * 1024, // 1MB
            IsExecutable = filePath.EndsWith(".exe") || filePath.EndsWith(".ps1"),
            ContainsSuspiciousStrings = filePath.Contains("malware") || filePath.Contains("hack"),
            Summary = $"File {filePath} analyzed. Executable: {filePath.EndsWith(".exe")}"
        };

        Console.WriteLine($"[StaticAnalysis] Suspicious: {result.IsSuspicious}");

        await context.EmitEventAsync(new KernelProcessEvent 
        { 
            Id = ProcessEvents.StaticAnalysisCompleted, 
            Data = result 
        });
    }
}

// Step 2: Decision Step (No AI)
public class DecisionStep : KernelProcessStep
{
    public static class Functions { public const string Decide = nameof(DecideAsync); }

    [KernelFunction(Functions.Decide)]
    public async Task DecideAsync(KernelProcessStepContext context, StaticAnalysisResult result)
    {
        Console.WriteLine("[Decision] Evaluating static analysis results...");

        if (result.IsSuspicious)
        {
            Console.WriteLine("[Decision] ⚠️ Suspicious activity detected. Routing to AI Classifier.");
            await context.EmitEventAsync(new KernelProcessEvent 
            { 
                Id = ProcessEvents.NeedsAIAnalysis, 
                Data = result 
            });
        }
        else
        {
            Console.WriteLine("[Decision] ✅ Low risk. Skipping AI classification.");
            await context.EmitEventAsync(new KernelProcessEvent 
            { 
                Id = ProcessEvents.SkipAIAnalysis, 
                Data = result 
            });
        }
    }
}

// Step 3: AI Malware Classification
public class AiClassificationStep : KernelProcessStep
{
    public static class Functions { public const string Classify = nameof(ClassifyAsync); }

    [KernelFunction(Functions.Classify)]
    public async Task ClassifyAsync(KernelProcessStepContext context, Kernel kernel, StaticAnalysisResult staticResult)
    {
        Console.WriteLine("[AI Classifier] Analyzing suspicious file metadata...");

        var chat = kernel.GetRequiredService<IChatCompletionService>();
        var prompt = $"""
            You are a malware analysis expert. Analyze the following static analysis report and provide a verdict.
            
            File: {staticResult.FilePath}
            Executable: {staticResult.IsExecutable}
            Suspicious Strings: {staticResult.ContainsSuspiciousStrings}
            Summary: {staticResult.Summary}

            Provide your output in this format:
            Verdict: [Malicious/Benign/Unknown]
            Confidence: [High/Medium/Low]
            Reasoning: [One sentence reasoning]
            """;

        var response = await chat.GetChatMessageContentAsync(prompt);
        var content = response.Content;

        // Simple parsing for demo purposes (in production, use Structured Output or JSON mode)
        var classification = new MalwareClassification
        {
            FilePath = staticResult.FilePath,
            Verdict = content.Contains("Malicious", StringComparison.OrdinalIgnoreCase) ? "Malicious" : "Benign",
            Confidence = "High",
            Reasoning = content.Replace("\n", " ").Trim()
        };

        Console.WriteLine($"[AI Classifier] Verdict: {classification.Verdict} | Confidence: {classification.Confidence}");

        await context.EmitEventAsync(new KernelProcessEvent 
        { 
            Id = ProcessEvents.AIClassified, 
            Data = classification 
        });
    }
}

// Step 4: AI Remediation Guidance
public class AiRemediationStep : KernelProcessStep
{
    public static class Functions 
    {
        public const string RecommendFromStatic = nameof(RecommendFromStaticAsync);
        public const string RecommendFromClassification = nameof(RecommendFromClassificationAsync);
    }

    [KernelFunction(Functions.RecommendFromStatic)]
    public async Task RecommendFromStaticAsync(KernelProcessStepContext context, Kernel kernel, StaticAnalysisResult result)
    {
        Console.WriteLine("[Remediation] Generating standard hygiene recommendations (Low Risk)...");
        // In a real scenario, we might still ask AI for a polite message or standard checks
        Console.WriteLine($"[Remediation] Action: No immediate threat found for {result.FilePath}. Standard monitoring applies.");
        await context.EmitEventAsync(new KernelProcessEvent { Id = ProcessEvents.RemediationGenerated });
    }

    [KernelFunction(Functions.RecommendFromClassification)]
    public async Task RecommendFromClassificationAsync(KernelProcessStepContext context, Kernel kernel, MalwareClassification classification)
    {
        Console.WriteLine("[Remediation] Generating targeted incident response plan...");
        
        var chat = kernel.GetRequiredService<IChatCompletionService>();
        var prompt = $"""
            The file '{classification.FilePath}' was classified as {classification.Verdict}.
            Reasoning: {classification.Reasoning}
            
            Generate 3 bullet points for immediate remediation or containment steps.
            """;

        var response = await chat.GetChatMessageContentAsync(prompt);
        Console.WriteLine($"[Remediation] Plan:\n{response.Content}");
        
        await context.EmitEventAsync(new KernelProcessEvent { Id = ProcessEvents.RemediationGenerated });
    }
}

## Part 3: Building and Running the Automated Pipeline

Now we wire up the steps using `ProcessBuilder`.

In [39]:
#pragma warning disable SKEXP0080

var processBuilder = new ProcessBuilder("MalwareAnalysisPipeline");

// Add steps
var staticStep = processBuilder.AddStepFromType<StaticAnalysisStep>();
var decisionStep = processBuilder.AddStepFromType<DecisionStep>();
var aiClassifierStep = processBuilder.AddStepFromType<AiClassificationStep>();
var aiRemediatorStep = processBuilder.AddStepFromType<AiRemediationStep>();

// 1. Start -> Static Analysis
processBuilder
    .OnInputEvent(ProcessEvents.AnalyzeFile)
    .SendEventTo(new ProcessFunctionTargetBuilder(staticStep, StaticAnalysisStep.Functions.RunStaticAnalysis, parameterName: "filePath"));

// 2. Static Analysis -> Decision
staticStep
    .OnEvent(ProcessEvents.StaticAnalysisCompleted)
    .SendEventTo(new ProcessFunctionTargetBuilder(decisionStep, DecisionStep.Functions.Decide, parameterName: "result"));

// 3A. Decision (Suspicious) -> AI Classifier
decisionStep
    .OnEvent(ProcessEvents.NeedsAIAnalysis)
    .SendEventTo(new ProcessFunctionTargetBuilder(aiClassifierStep, AiClassificationStep.Functions.Classify, parameterName: "staticResult"));

// 3B. Decision (Not Suspicious) -> Remediation (Static path)
decisionStep
    .OnEvent(ProcessEvents.SkipAIAnalysis)
    .SendEventTo(new ProcessFunctionTargetBuilder(aiRemediatorStep, AiRemediationStep.Functions.RecommendFromStatic, parameterName: "result"));

// 4. AI Classifier -> Remediation (Classification path)
aiClassifierStep
    .OnEvent(ProcessEvents.AIClassified)
    .SendEventTo(new ProcessFunctionTargetBuilder(aiRemediatorStep, AiRemediationStep.Functions.RecommendFromClassification, parameterName: "classification"));

var process = processBuilder.Build();

Console.WriteLine("Pipeline built successfully.");

Pipeline built successfully.


In [40]:
#pragma warning disable SKEXP0080

// Run the process with a "Suspicious" file
Console.WriteLine("--- Running Pipeline for Suspicious File ---");
await process.StartAsync(kernel, new KernelProcessEvent { Id = ProcessEvents.AnalyzeFile, Data = "suspicious_script.ps1" });

Console.WriteLine("\n--- Running Pipeline for Benign File ---");
await process.StartAsync(kernel, new KernelProcessEvent { Id = ProcessEvents.AnalyzeFile, Data = "notes.txt" });

--- Running Pipeline for Suspicious File ---

[StaticAnalysis] Analyzing file: suspicious_script.ps1
[StaticAnalysis] Suspicious: True
[Decision] Evaluating static analysis results...
[Decision] ⚠️ Suspicious activity detected. Routing to AI Classifier.
[AI Classifier] Analyzing suspicious file metadata...
[AI Classifier] Verdict: Malicious | Confidence: High
[Remediation] Generating targeted incident response plan...
[Remediation] Plan:
Certainly! Here are three remediation or containment steps:  

1. **Isolate the Affected System**: Immediately isolate the system where the suspicious PowerShell script (`suspicious_script.ps1`) was detected from the network to prevent lateral movement or further exploitation.  

2. **Analyze the Script**: Conduct a detailed manual or automated analysis of the script to identify any possible malicious behavior, including contacting any domains, executing commands, or modifying critical files. Use tools such as a sandbox or static code analyzer to exami

## Part 4: Adding Human-in-the-Loop (HITL)

In high-stakes security scenarios, we often don't want AI to trigger automated remediation (like isolating a CEO's laptop) without human verification.

We will modify the pipeline to inject a **Human Review Step** after the AI Classification but before Remediation.

In [41]:
#pragma warning disable SKEXP0080

public class HumanReviewStep : KernelProcessStep
{
    public static class Functions { public const string Review = nameof(ReviewAsync); }

    [KernelFunction(Functions.Review)]
    public async Task ReviewAsync(KernelProcessStepContext context, MalwareClassification classification)
    {
        Console.WriteLine("\n[Human Review] 👮 STOP! Human verification required.");
        Console.WriteLine($"File: {classification.FilePath}");
        Console.WriteLine($"AI Verdict: {classification.Verdict}");
        Console.WriteLine($"Reasoning: {classification.Reasoning}");

        // In a real app, this would wait for an external event (API call, button click)
        var userInput = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync($"Human, You are in the Loop, AI Verdict: {classification.Verdict} Reasoning: {classification.Reasoning}, do you approve (True/False)????");
        bool approved = bool.Parse(userInput);

        if (approved)
        {
            Console.WriteLine("[Human Review] ✅ Analyst CONFIRMED verdict. Proceeding to remediation.");
            await context.EmitEventAsync(new KernelProcessEvent 
            { 
                Id = ProcessEvents.HumanVerified, 
                Data = classification 
            });
        }
        else
        {
            Console.WriteLine("[Human Review] ❌ Analyst REJECTED verdict. Stopping process.");
            await context.EmitEventAsync(new KernelProcessEvent { Id = ProcessEvents.HumanRejected });
        }
    }
}

In [42]:
#pragma warning disable SKEXP0080

// Re-build the process with HITL wiring
var hitlBuilder = new ProcessBuilder("MalwareAnalysisPipeline_HITL");

var staticStep2 = hitlBuilder.AddStepFromType<StaticAnalysisStep>();
var decisionStep2 = hitlBuilder.AddStepFromType<DecisionStep>();
var aiClassifierStep2 = hitlBuilder.AddStepFromType<AiClassificationStep>();
var humanReviewStep = hitlBuilder.AddStepFromType<HumanReviewStep>(); // New Step
var aiRemediatorStep2 = hitlBuilder.AddStepFromType<AiRemediationStep>();

// 1. Start -> Static
hitlBuilder.OnInputEvent(ProcessEvents.AnalyzeFile)
    .SendEventTo(new ProcessFunctionTargetBuilder(staticStep2, StaticAnalysisStep.Functions.RunStaticAnalysis, parameterName: "filePath"));

// 2. Static -> Decision
staticStep2.OnEvent(ProcessEvents.StaticAnalysisCompleted)
    .SendEventTo(new ProcessFunctionTargetBuilder(decisionStep2, DecisionStep.Functions.Decide, parameterName: "result"));

// 3A. Decision (Suspicious) -> AI Classifier
decisionStep2.OnEvent(ProcessEvents.NeedsAIAnalysis)
    .SendEventTo(new ProcessFunctionTargetBuilder(aiClassifierStep2, AiClassificationStep.Functions.Classify, parameterName: "staticResult"));

// 3B. Decision (Not Suspicious) -> Remediation (Direct)
decisionStep2.OnEvent(ProcessEvents.SkipAIAnalysis)
    .SendEventTo(new ProcessFunctionTargetBuilder(aiRemediatorStep2, AiRemediationStep.Functions.RecommendFromStatic, parameterName: "result"));

// 4. AI Classifier -> HUMAN REVIEW (Changed from Direct Remediation)
aiClassifierStep2.OnEvent(ProcessEvents.AIClassified)
    .SendEventTo(new ProcessFunctionTargetBuilder(humanReviewStep, HumanReviewStep.Functions.Review, parameterName: "classification"));

// 5. Human Verified -> Remediation
humanReviewStep.OnEvent(ProcessEvents.HumanVerified)
    .SendEventTo(new ProcessFunctionTargetBuilder(aiRemediatorStep2, AiRemediationStep.Functions.RecommendFromClassification, parameterName: "classification"));

// 6. Human Rejected -> Stop (or route elsewhere)
humanReviewStep.OnEvent(ProcessEvents.HumanRejected)
    .StopProcess();

var hitlProcess = hitlBuilder.Build();

Console.WriteLine("HITL Pipeline built.");

HITL Pipeline built.


In [43]:
#pragma warning disable SKEXP0080

// Run the HITL process
Console.WriteLine("--- Running HITL Pipeline ---");
await hitlProcess.StartAsync(kernel, new KernelProcessEvent { Id = ProcessEvents.AnalyzeFile, Data = "critical_malware.exe" });

--- Running HITL Pipeline ---

[StaticAnalysis] Analyzing file: critical_malware.exe
[StaticAnalysis] Suspicious: True
[Decision] Evaluating static analysis results...
[Decision] ⚠️ Suspicious activity detected. Routing to AI Classifier.
[AI Classifier] Analyzing suspicious file metadata...
[AI Classifier] Verdict: Malicious | Confidence: High

[Human Review] 👮 STOP! Human verification required.
File: critical_malware.exe
AI Verdict: Malicious
Reasoning: Verdict: Malicious   Confidence: Medium   Reasoning: The file is executable and contains suspicious strings, both strong indicators of potentially malicious behavior, but further dynamic analysis is needed for confirmation.
[Human Review] ✅ Analyst CONFIRMED verdict. Proceeding to remediation.
[Remediation] Generating targeted incident response plan...
[Remediation] Plan:
- **Isolate the Affected System:** Immediately disconnect the system containing 'critical_malware.exe' from the network to avoid potential lateral movement or further